In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install fastparquet
!pip install --upgrade numpy

     |████████████████████████████████| 1.2 MB 4.0 MB/s 
     |████████████████████████████████| 125 kB 53.3 MB/s 
     |████████████████████████████████| 59 kB 6.0 MB/s 
     |████████████████████████████████| 1.5 MB 40.3 MB/s 
  Created wheel for thrift: filename=thrift-0.15.0-cp37-cp37m-linux_x86_64.whl size=348201 sha256=1bbea8c0db1792b32ca04894274437607d6093ca079b85c2106baa85120d5517
  Stored in directory: /root/.cache/pip/wheels/ba/1f/8e/e6fd36837eecf3d1f2b23f1729477e8e06558d8d60b7093f51
Successfully built thrift
     |████████████████████████████████| 15.7 MB 121 kB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.6.0 requires numpy~=1.19.2, but you have numpy 1.21.3 which is incompatible.
d

In [3]:
import os
import re
import numpy as np
import pandas as pd
import fastparquet as fp

os.chdir('/content/gdrive/My Drive/phyto_curves_reco')
from pred_functions import combine_files_into_acquisitions, post_processing

# Fetch the files 
os.chdir('/content/gdrive/My Drive/Results/SSLAMM/')
#pred_folders =  ['preds1', 'preds2']

pulse_regex = "Pulse" 
date_regex = "Pulse[0-9]{1,2}_(20[0-9]{2}-[0-9]{2}-[0-9]{2} [0-9]{2}(?:u|h)[0-9]{2})"
flr_regex = 'Pulse([0-9]{1,2})' #flr5 ou flr25

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [4]:
phyto_ts = pd.DataFrame(columns = ['MICRO', 'ORGNANO', 'ORGPICOPRO', 'REDNANO', 'REDPICOEUK',\
                   'REDPICOPRO', 'inf1microm','sup1microm', 'date', 'FLR', 'file'])

In [13]:
pred_folder = 'Preds1'
pred_files = os.listdir(pred_folder)
files = [file for file in pred_files if re.search(pulse_regex, file)] # The files containing the data to predict

In [16]:
phyto_ts['file']

Pred PFG name    Pulse6_2019-09-24 13h59.parq
Pred PFG name    Pulse6_2019-11-08 15h59.parq
Pred PFG name    Pulse6_2019-11-16 17h59.parq
Pred PFG name    Pulse6_2019-10-10 01h59.parq
Pred PFG name    Pulse6_2019-11-16 19h59.parq
                             ...             
Pred PFG name    Pulse6_2019-11-08 11h59.parq
Pred PFG name    Pulse6_2019-11-08 12h58.parq
Pred PFG name    Pulse6_2019-11-16 15h59.parq
Pred PFG name    Pulse6_2019-11-08 13h59.parq
Pred PFG name    Pulse6_2019-11-27 15h59.parq
Name: file, Length: 1519, dtype: object

True

In [ ]:
for file in files:
  if file in phyto_ts['file'].to_list():
    print('bla')
    continue

In [21]:
for file in files:

    if file in phyto_ts['file'].to_list():
      continue

    flr_num = int(re.search(flr_regex, file).group(1))
    path = pred_folder + '/' + file
    pfile = fp.ParquetFile(path)
    cl_count = pfile.to_pandas(columns=['Pred PFG name'])['Pred PFG name'].value_counts()  #, 'Total FWS', 'Total SWS'
    cl_count = pd.DataFrame(cl_count).transpose() #met sous forme de tableau+échange lignes et colonnes
        
    date = re.search(date_regex, file).group(1)  
    
    # The timestamp is rounded to the closest hour
    date = pd.to_datetime(date, format='%Y-%m-%d %Hh%M', errors='ignore')
    date = date.round('2H')
           
    cl_count['date'] = date #ce qu'on met dans la colonne date
    cl_count['FLR'] = flr_num 
    cl_count['file'] = file 

    # New format:
    phyto_ts = phyto_ts.append(cl_count)


In [23]:
phyto_ts.to_csv('preds20211027_full_SSLAMM.csv', index = False)

In [ ]:
from copy import deepcopy
phyto_ts_backup = deepcopy(phyto_ts)
phyto_ts = combine_files_into_acquisitions(phyto_ts.set_index('file')) #les prédictions FLR5 et 20 sont combinées

In [ ]:
phyto_ts.fillna(0).to_csv('../results/preds20211021_combined.csv', index = False)

# Deal with incomplete acquisitions

In [ ]:
phyto_ts.to_csv('/content/gdrive/My Drive/SSLAMM_preds20211021_retreated.csv', index = False)

In [ ]:
#phyto_ts.to_csv('/content/gdrive/My Drive/data/SWINGS/SWINGS_preds06212021.csv')